# Flight Fare Prediction - Starter Notebook

In [ ]:
!pip install scikit-learn


## 🧹 1. Import Libraries and Load Dataset

In [1]:
#1. Import Libraries and Load Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Load dataset
df = pd.read_excel("flight_fare.xlsx")
df.head()


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


## 🔍 2. Data Cleaning & Preprocessing

In [3]:
#2. Data Cleaning & Preprocessing
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

       Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  Price  
0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897  
1    05:50         13:15   7h 25m     2 stops         No info   7662  
2    09:25  04:25 10 Jun      19h     2 stops         No info  13882  
3    18:05         23:30   5h 25m      1 stop         No info   6218  
4    16:50         21:35   4h 45m      1 stop         No info  13302  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 

In [42]:
'''
Data cleaning
1) remove the rows with null values
2) change total stop to integer 
3) convert duration to just hours and the to integer
4) deperature time is inthe format hour:min
5) arrival time is in the format of hour:min [date](if it arrives on the next day)
6) so is there a good way to convert arrival time and departure time to compare them
'''

""" 1)Analyzing when will most flights take off
    2)Analyzing whether flight duration impacts price or not
    3)Analyzing how the no. of stops impacts price
    4)Finding which airline has the highest price
    5)Finding mean prices for airlines
"""

' 1)Analyzing when will most flights take off\n    2)Analyzing whether flight duration impacts price or not\n    3)Analyzing how the no. of stops impacts price\n    4)Finding which airline has the highest price\n    5)Finding mean prices for airlines\n'

In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


In [5]:
df['Total_Stops'] = df['Total_Stops'].replace({
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4
})
df['Total_Stops'] = df['Total_Stops'].astype(int)


C:\Users\nageshbhagelli\AppData\Local\Temp\ipykernel_5896\3292887850.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Total_Stops'] = df['Total_Stops'].replace({


In [6]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1,No info,13302


In [7]:
def get_decimal_hours(duration_str):
    if 'h' not in duration_str:
        duration_str = '0h ' + duration_str
    if 'm' not in duration_str:
        duration_str = duration_str + ' 0m'
    h, m = duration_str.replace('h', '').replace('m', '').split()
    return int(h) + int(m) / 60

df['Duration_Hour_Decimal'] = df['Duration'].apply(get_decimal_hours)


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Airline                10682 non-null  object 
 1   Date_of_Journey        10682 non-null  object 
 2   Source                 10682 non-null  object 
 3   Destination            10682 non-null  object 
 4   Route                  10682 non-null  object 
 5   Dep_Time               10682 non-null  object 
 6   Arrival_Time           10682 non-null  object 
 7   Duration               10682 non-null  object 
 8   Total_Stops            10682 non-null  int64  
 9   Additional_Info        10682 non-null  object 
 10  Price                  10682 non-null  int64  
 11  Duration_Hour_Decimal  10682 non-null  float64
dtypes: float64(1), int64(2), object(9)
memory usage: 1.1+ MB


In [48]:
# Step 1: Parse time columns without date
df['Dep_Time_parsed'] = pd.to_datetime(df['Dep_Time'], format='%H:%M')
df['Arrival_Time_parsed'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M', errors='coerce')

wrong_same_day = df[df['Arrival_Time_parsed'] < df['Dep_Time_parsed']]
# Step 2: Mark suspicious timings
df['Suspicious_Timing'] = df['Arrival_Time_parsed'] < df['Dep_Time_parsed']

# Step 3: Drop them
df.drop(index=df[df["Suspicious_Timing"]].index, inplace=True)

# Optional: Reset index
df.reset_index(drop=True, inplace=True)


In [8]:

df.drop(columns=['Dep_Time_parsed', 'Arrival_Time_parsed', 'Suspicious_Timing'], inplace=True)
df.head()


KeyError: "['Dep_Time_parsed', 'Arrival_Time_parsed', 'Suspicious_Timing'] not found in axis"

In [2]:
df_copy=df.copy()
df_copy.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [56]:
# Parse Date_of_Journey
df_copy['Journey_Date'] = pd.to_datetime(df_copy['Date_of_Journey'], format='%d/%m/%Y')

# Function to parse arrival date from Arrival_Time column
def get_arrival_date(arrival_str, journey_year):
    parts = arrival_str.split(' ')
    if len(parts) > 1:
        # Extract day and month from Arrival_Time string
        date_str = parts[1] + ' ' + parts[2] if len(parts) > 2 else parts[1]
        # Construct datetime for arrival date with the journey year
        return pd.to_datetime(f"{journey_year} {date_str}", format='%Y %d %b')
    else:
        # No date part, assume arrival on journey date
        return None

# Apply function to get arrival date (only date, not time)
df_copy['Arrival_Date'] = df_copy.apply(lambda row: get_arrival_date(row['Arrival_Time'], row['Journey_Date'].year), axis=1)

# For rows where Arrival_Date is missing (no date in Arrival_Time), fill with Journey_Date
df_copy['Arrival_Date'].fillna(df_copy['Journey_Date'], inplace=True)

df_copy.loc[df_copy['Arrival_Date'] < df_copy['Journey_Date']]


C:\Users\Owais\AppData\Local\Temp\ipykernel_8496\3304908666.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Arrival_Date'].fillna(df_copy['Journey_Date'], inplace=True)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Duration_Hour_Decimal,Journey_Date,Arrival_Date
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,No info,3897,2.833333,2019-03-24,2019-03-22
21,Air India,21/03/2019,Banglore,New Delhi,BLR → COK → DEL,22:00,13:20 19 Mar,15h 20m,1,No info,6955,15.333333,2019-03-21,2019-03-19
110,Jet Airways,18/03/2019,Banglore,New Delhi,BLR → BOM → DEL,14:05,05:05 16 Mar,15h,1,No info,13555,15.000000,2019-03-18,2019-03-16
379,Jet Airways,18/03/2019,Banglore,New Delhi,BLR → BOM → DEL,16:55,09:00 16 Mar,16h 5m,1,In-flight meal not included,11087,16.083333,2019-03-18,2019-03-16
605,Air India,21/03/2019,Banglore,New Delhi,BLR → COK → DEL,22:00,10:50 19 Mar,12h 50m,1,No info,6955,12.833333,2019-03-21,2019-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10437,Jet Airways,18/03/2019,Banglore,New Delhi,BLR → BOM → DEL,11:40,08:15 16 Mar,20h 35m,1,No info,12284,20.583333,2019-03-18,2019-03-16
10485,IndiGo,15/03/2019,Chennai,Kolkata,MAA → CCU,22:05,00:30 13 Mar,2h 25m,0,No info,6297,2.416667,2019-03-15,2019-03-13
10586,Air Asia,24/03/2019,Banglore,New Delhi,BLR → DEL,23:25,02:10 22 Mar,2h 45m,0,No info,3383,2.750000,2019-03-24,2019-03-22
10634,Jet Airways,21/03/2019,Banglore,New Delhi,BLR → BOM → DEL,21:25,11:25 19 Mar,14h,1,In-flight meal not included,7832,14.000000,2019-03-21,2019-03-19


In [57]:
# Drop rows where arrival date is before journey date
df_copy = df_copy[df_copy['Arrival_Date'] >= df_copy['Journey_Date']]

# Optional: reset index
df_copy.reset_index(drop=True, inplace=True)
df_copy.loc[df_copy['Arrival_Date'] < df_copy['Journey_Date']]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Duration_Hour_Decimal,Journey_Date,Arrival_Date


In [3]:
df_copy.info()
df=df_copy.copy()
df.to_csv('Flight.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [73]:
#Machine Learning Algorithm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 1: Create a copy of the cleaned DataFrame
df_ml = df.copy()
# Step 2: Ensure Duration is numeric (you've already done this, so skip if Duration_mins exists)
# Step 3: Convert Total_Stops to numeric
df_ml['Total_Stops'] = df_ml['Total_Stops'].replace({
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4
}).astype(int)
# Step 4: Select features and target
features = ['Airline', 'Source', 'Destination', 'Total_Stops']
X = df_ml[features]
y = df_ml['Price']
# Step 5: One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['Airline', 'Source', 'Destination'], drop_first=True)
# Step 6: Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
# Step 7: Train Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
# Step 8: Predict and evaluate
y_pred = model.predict(X_test)
print("Model Evaluation:")
print("MAE :", mean_absolute_error(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))        # ✅
print("R² Score:", r2_score(y_test, y_pred))


📊 Model Evaluation:
MAE : 1790.2541378966284
MSE : 7076026.808572815
RMSE: 2660.080225965528
R² Score: 0.6833134141466504


In [77]:
def predict_price(model, training_columns, airline, source, destination, stops):
    import pandas as pd
    # Step 1: Create a new sample input
    input_df = pd.DataFrame({
        'Airline': [airline],
        'Source': [source],
        'Destination': [destination],
        'Total_Stops': [stops]
    })
    # Step 2: One-hot encode like training set
    input_encoded = pd.get_dummies(input_df)

    # Step 3: Reindex to match training data columns
    input_encoded = input_encoded.reindex(columns=training_columns, fill_value=0)
    # Step 4: Predict
    prediction = model.predict(input_encoded)
    return prediction[0]
predicted_price = predict_price(
    model=model,
    training_columns=X_train.columns,  # Now it's clearly column names
    airline='Jet Airways',
    source='Delhi',
    destination='Cochin',
    stops=1
)
print(f"Predicted Price: ₹{round(predicted_price, 2)}")

Predicted Price: ₹12402.85


In [80]:
df.loc[(df["Destination"]=='Cochin') & (df["Airline"]=="Jet Airways") & (df["Total_Stops"]==1)]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Duration_Hour_Decimal,Journey_Date,Arrival_Date
16,Jet Airways,12/06/2019,Delhi,Cochin,DEL → BOM → COK,14:00,12:35 13 Jun,22h 35m,1,In-flight meal not included,10262,22.583333,2019-06-12,2019-06-13
18,Jet Airways,27/05/2019,Delhi,Cochin,DEL → BOM → COK,16:00,12:35 28 May,20h 35m,1,In-flight meal not included,12898,20.583333,2019-05-27,2019-05-28
32,Jet Airways,18/05/2019,Delhi,Cochin,DEL → BOM → COK,07:05,12:35,5h 30m,1,In-flight meal not included,12373,5.500000,2019-05-18,2019-05-18
37,Jet Airways,3/06/2019,Delhi,Cochin,DEL → BOM → COK,07:05,19:00,11h 55m,1,No info,14924,11.916667,2019-06-03,2019-06-03
40,Jet Airways,18/05/2019,Delhi,Cochin,DEL → BOM → COK,20:55,19:00 19 May,22h 5m,1,In-flight meal not included,12373,22.083333,2019-05-18,2019-05-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10505,Jet Airways,1/06/2019,Delhi,Cochin,DEL → BOM → COK,10:00,12:35 02 Jun,26h 35m,1,In-flight meal not included,10262,26.583333,2019-06-01,2019-06-02
10507,Jet Airways,9/05/2019,Delhi,Cochin,DEL → BOM → COK,18:15,19:00 10 May,24h 45m,1,In-flight meal not included,12373,24.750000,2019-05-09,2019-05-10
10544,Jet Airways,1/06/2019,Delhi,Cochin,DEL → BOM → COK,22:50,12:35 02 Jun,13h 45m,1,No info,14714,13.750000,2019-06-01,2019-06-02
10548,Jet Airways,9/03/2019,Delhi,Cochin,DEL → BOM → COK,18:15,18:50 10 Mar,24h 35m,1,In-flight meal not included,14053,24.583333,2019-03-09,2019-03-10
